In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import pickle


# Load Data

In [ ]:
folder = Path("Convolution_Pairs/")
########################################### Load
#################
################
for file in folder.glob("*.pkl"):
    

    
    #save_name = "Convolution_Pairs/"+str(ID) + "_Convolution_Pair.pkl"
    
    # load
    with open(file, "rb") as f:
        data_pair = pickle.load(f)

    normailization_tensor_img,lis  = data_pair
    INC_MORPH_PETR, INC_MORPH_SERSIC, INC_KIN = lis
    

# Make netowrk

In [ ]:
# ----------------------
# 1. Data Loading
# ----------------------

#Load 

folder = Path("Convolution_Pairs/")



    
for file in folder.glob("*.npy"):

    
    print(file)
    convolution,incliniation  = np.load(file)
    

# ----------------------
# 3. Training
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train for 1 epoch (for demo)
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 200 == 0:
        print(f"Train Step {batch_idx}, Loss: {loss.item():.4f}")
    if batch_idx == 500:  # stop early for speed
        break

# ----------------------
# 4. Evaluation
# ----------------------
model.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

print(f"Test Accuracy: {correct / len(test_loader.dataset):.2%}")


In [ ]:
#FRom https://github.com/ChawDoe/LeNet5-MNIST-PyTorch/blob/master/train.py

if __name__ == '__main__':
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    batch_size = 256
    train_dataset = mnist.MNIST(root='./train', train=True, transform=ToTensor())
    test_dataset = mnist.MNIST(root='./test', train=False, transform=ToTensor())
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    model = Model().to(device)
    sgd = SGD(model.parameters(), lr=1e-1)
    loss_fn = CrossEntropyLoss()
    all_epoch = 100
    prev_acc = 0
    for current_epoch in range(all_epoch):
        model.train()
        for idx, (train_x, train_label) in enumerate(train_loader):
            train_x = train_x.to(device)
            train_label = train_label.to(device)
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = loss_fn(predict_y, train_label.long())
            loss.backward()
            sgd.step()

        all_correct_num = 0
        all_sample_num = 0
        model.eval()
        
        for idx, (test_x, test_label) in enumerate(test_loader):
            test_x = test_x.to(device)
            test_label = test_label.to(device)
            predict_y = model(test_x.float()).detach()
            predict_y =torch.argmax(predict_y, dim=-1)
            current_correct_num = predict_y == test_label
            all_correct_num += np.sum(current_correct_num.to('cpu').numpy(), axis=-1)
            all_sample_num += current_correct_num.shape[0]
        acc = all_correct_num / all_sample_num
        print('accuracy: {:.3f}'.format(acc), flush=True)
        if not os.path.isdir("models"):
            os.mkdir("models")
        torch.save(model, 'models/mnist_{:.3f}.pkl'.format(acc))
        if np.abs(acc - prev_acc) < 1e-4:
            break
        prev_acc = acc
    print("Model finished training")

# Use transfer learning to reduce training time with pre-trained weights

In [2]:
import torch, torch.nn as nn
import torchvision.models as models
from torchvision import transforms
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Load pretrained model
model = models.resnet18(pretrained=True)

# 2) Freeze backbone
for param in model.parameters():
    param.requires_grad = False

# 3) Replace head for single-output regression
in_feats = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(in_feats, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1)   # single continuous output
)

#model = model.to(device)


# 4) Only params in model.fc will have requires_grad True
optimizer = torch.optim.AdamW(model.fc.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.MSELoss()


/home/ben/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 750 which is of cuda capability 5.0.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/home/ben/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/home/ben/miniconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 750 with CUDA capability sm_50 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 750 GPU with PyTorch, please check the instructions at https://pytorch.org/ge